In [1]:
import librosa
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import pickle
from sklearn.model_selection import train_test_split

In [2]:
def extract_mfcc(filepath, n_mfcc=13, hop_length=512, n_fft=2048):
    if filepath.endswith('.wav'):
        audio, sr = librosa.load(filepath, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
        features = mfcc.T #Transpose to get the correct shape
        average_features = np.mean(features, axis=0)
        return average_features

In [3]:
train_folder_path = '/content/drive/MyDrive/Assignment 2/train'
mfcc_features_list = []
file_names_list = []

for filename in os.listdir(train_folder_path):
    filepath = os.path.join(train_folder_path, filename)
    if filepath.endswith('.wav'):
        mfcc = extract_mfcc(filepath)
        if mfcc is not None:
            mfcc_features_list.append(mfcc)
            file_names_list.append(filename)

# Convert the MFCC features list to a NumPy array
mfcc_features_array = np.array(mfcc_features_list)

In [4]:
mfcc_train, mfcc_val, filenames_train, filenames_val = train_test_split(
    mfcc_features_array,
    file_names_list,
    test_size=0.2,
    random_state=42
)

In [5]:
# Save the filenames list as a pickle file
with open('/content/drive/MyDrive/train_filenames.pkl', 'wb') as f:
    pickle.dump(file_names_list, f)

In [6]:
print(mfcc_features_array[0])

[-1.8044212e+02  1.3999611e+02 -8.1985598e+00  3.1531612e+01
 -4.2815495e+00  2.0610485e+01 -1.0084117e+01  1.3115306e+01
  5.2859783e-01  2.1011593e+00  1.3066685e-01  3.4481905e+00
  5.4831678e-01]


In [7]:
print(f"MFCC Features Array Shape: {mfcc_features_array.shape}")

MFCC Features Array Shape: (220, 13)


In [8]:
input_shape = mfcc_features_array.shape[1]

# Encoder
inputs = Input(shape=(input_shape))
encoded = Dense(128, activation="relu")(inputs)
encoded = Dense(64, activation="relu")(encoded)
encoded = Dense(32, activation="relu", name="encoder_layer")(encoded)

# Decoder
decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(128, activation="relu")(decoded)
decoded = Dense(13)(decoded)

autoencoder = Model(inputs, decoded)

In [9]:
# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
autoencoder.fit(mfcc_train, mfcc_train, epochs=100, batch_size=32, validation_data=(mfcc_val, mfcc_val))

Epoch 1/100
6/6 [==============================] - 7s 58ms/step - loss: 5360.7964 - val_loss: 5454.2563
Epoch 2/100
6/6 [==============================] - 0s 12ms/step - loss: 4039.1040 - val_loss: 3329.0596
Epoch 3/100
6/6 [==============================] - 0s 10ms/step - loss: 2091.8552 - val_loss: 1194.9911
Epoch 4/100
6/6 [==============================] - 0s 8ms/step - loss: 1064.3538 - val_loss: 935.5355
Epoch 5/100
6/6 [==============================] - 0s 8ms/step - loss: 812.5842 - val_loss: 613.2629
Epoch 6/100
6/6 [==============================] - 0s 9ms/step - loss: 706.9299 - val_loss: 611.8807
Epoch 7/100
6/6 [==============================] - 0s 8ms/step - loss: 642.0526 - val_loss: 440.3339
Epoch 8/100
6/6 [==============================] - 0s 11ms/step - loss: 545.2529 - val_loss: 386.0972
Epoch 9/100
6/6 [==============================] - 0s 8ms/step - loss: 476.2526 - val_loss: 322.6320
Epoch 10/100
6/6 [==============================] - 0s 9ms/step - loss: 405.7519

In [10]:
# Extract the encoder part of the model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoder_layer').output)

# Encode all training audios to latent representations
latent_representations = encoder.predict(mfcc_features_array)

7/7 [==============================] - 0s 2ms/step


In [11]:
# Save the latent representations (LRDB) as a pickle file
with open('/content/drive/MyDrive/LRDB.pkl', 'wb') as f:
    pickle.dump(latent_representations, f)

In [12]:
encoder.save('/content/drive/MyDrive/encoder_model.keras')